In [ ]:
# Get reference to the model
from azureml.core import Workspace, Model

ws = Workspace.from_config()

model = Model(ws, 'automl_diabetes')

In [ ]:
# Download latest artifacts, overriding them
model.download('./', exist_ok= True)

In [ ]:
from azureml.core import Environment

myEnv = Environment.from_conda_specification("automl-inference",'outputs/conda_env_v_1_0_0.yml')

# Assing environment variable to locate the name. Could also be found spliting the 
# AZUREML_MODEL_DIR environment variable
myEnv.environment_variables['model_name'] = model.name

In [ ]:
from azureml.core.webservice import AciWebservice

aci_deployment_config = AciWebservice.deploy_configuration(
    description=model.description,
    cpu_cores=1,
    memory_gb= 1.5,
    auth_enabled= True,
    dns_name_label="funwithautoml",
    tags=model.tags,
    properties={'CreatedBy':'FastTrack for Azure'}
)

In [ ]:
from azureml.core.model import InferenceConfig
#https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py

inference_config = InferenceConfig(
    entry_script="custom_scoring.py",
    source_directory=".",
    environment= myEnv
)

In [ ]:
service = Model.deploy(
    workspace=ws,
    name="fun-with-automl",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_deployment_config,
    overwrite=True
)

In [ ]:
service.wait_for_deployment(show_output=True)